# SAP Cloud ALM Process Scopes API Test and Demo

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for implementation. The specific APIs shown below are for Process Scopes.

The API information and specification is available here:

  - https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
  - https://api.sap.com/api/CALM_PM/overview - SAP Cloud ALM Process Scopes

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples.

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

  - Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support
  - Python 3, a recent version, is Required. Python 3.9 was used here
  - Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://requests.readthedocs.io/en/latest/user/install/#install
  - Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
  - Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html

## APIs called

API for Process Scopes: https://\<tenanant url\>/api/calm-processmanagement/v1

* * *
# Authentication information

You must create a python module file called apidata.py and put the information specific to your tenant there. This includes:

  - OAuth2 client ID and secret
  - Token url
  - Base URL for API calls
  
Get client ID and secret variables from an external module: this information is sensitive.

These items can be retrieved from the SAP Business Technology Platform (SAP BTP) Cockpit.

## Format of module apidata.py for import

```python
service_instance_client_id = r'get your client ID from SAP BTP Cockpit'
service_instance_client_secret = r'get your client secret from SAP BTP Cockpit'
token_url = 'your token url'
base_url = 'your base url'
```

In [ ]:
# Import credentials from apidata.py
import apidata as ad

client_id = ad.service_instance_client_id
client_secret = ad.service_instance_client_secret
token_url = ad.token_url
base_url = ad.base_url

default_project = '11111111-1111-1111-1111-111111111111'

* * *
# Get token for authentication
Call OAuth token API with credential information. Add the token to variable which is used as header in all subsequent requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import pandas as pd

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

# Prepare the header data for all subsequent requests
headers = {'Authorization': 'Bearer ' + token['access_token']}

# prepare the header for patch requests
patchUpdateHeader = {
    'Authorization': 'Bearer ' + token['access_token'],
    'Content-Type': 'application/merge-patch+json'
} 

* * * 
# GET count of all scopes

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/scopes/$count', headers=headers)

print(response.status_code, response.reason)
print('Number of scopes: ', response.text)

* * *
# GET a list of scopes in the default project

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/scopes/?projectId=' + default_project, headers=headers)

print(response.status_code, response.reason)

# Parse JSON response into Pandas Dataframe

This takes the data returned from the process scopes API, which is in JSON format, and places it into a dataframe for further processing and analysis.

In [ ]:
df = pd.json_normalize(response.json()['value'])

df

* * * 
# GET a list of scopes using paging parameters and inline count

Expected response: "200 OK"

In [ ]:
response = requests.get(base_url + '/scopes?$skip=2&$top=3&$count=true', headers=headers)

print(response.status_code, response.reason)

# Parse JSON response into Pandas Dataframe

This takes the data returned from the process scopes API, which is in JSON format, and places it into a dataframe for further processing and analysis.

In [ ]:
df = pd.json_normalize(response.json()['value'])

df

* * *
# Create a scope

Expected response "201 Created"

In [ ]:
import datetime

isoNow = datetime.datetime.now().isoformat()

scopeData = {
    'projectId': default_project,
    'name': 'API test scope ' + isoNow,
    'description': 'test description'
}

createdScopeResponse = requests.post(base_url + '/scopes', headers=headers, json=scopeData)

print(createdScopeResponse.status_code, createdScopeResponse.reason)

newScopeId = createdScopeResponse.json()['id']
print('New scope ID: ', newScopeId)

df = pd.json_normalize(createdScopeResponse.json())

df

* * *

# Update a scope

Expected response "200 OK"

In [ ]:
scopeUpdateData = {
    'name': scopeData['name'] + ' Updated',
    'description': scopeData['description'] + ' Updated'
}

updatedScopeResponse = requests.patch(base_url + '/scopes/' + newScopeId, headers=patchUpdateHeader, json=scopeUpdateData)

print(updatedScopeResponse.status_code, updatedScopeResponse.reason)

df = pd.json_normalize(updatedScopeResponse.json())

df

* * * 
# Get all solution scenario versions (content release versions)

Expected response "200 OK"

In [ ]:
allContentSolutionScenarioVersions = requests.get(base_url + '/solutionScenarioVersions', headers=headers)

print(allContentSolutionScenarioVersions.status_code, allContentSolutionScenarioVersions.reason)

df = pd.json_normalize(allContentSolutionScenarioVersions.json()['value'])

df

* * * 
# Assign a solution scenario version to a scope

Expected response "201 Created"

In [ ]:
solutionScenarioAssignment = {
    'value': [{
        'id': allContentSolutionScenarioVersions.json()['value'][10]['id']
    },
    {
        'id': allContentSolutionScenarioVersions.json()['value'][12]['id']
    }]
}

assignSolutionScenarioResponse = requests.post(base_url + '/scopes/' + newScopeId + '/solutionScenarioVersions', headers=headers, json=solutionScenarioAssignment)

print(assignSolutionScenarioResponse.status_code, assignSolutionScenarioResponse.reason)

df = pd.json_normalize(assignSolutionScenarioResponse.json()['value'])

df

* * *
# Get all assigned solution scenario versions from single scope

Expected response "200 OK"

In [ ]:
allContentSolutionScenarioVersionsFromSingleScope = requests.get(base_url + '/scopes/' + newScopeId + '/solutionScenarioVersions', headers=headers)

print(allContentSolutionScenarioVersionsFromSingleScope.status_code, allContentSolutionScenarioVersionsFromSingleScope.reason)

df = pd.json_normalize(allContentSolutionScenarioVersionsFromSingleScope.json()['value'])

df

* * *
# Get a list of solution processes for the default project and the created scope

Expected response "200 OK"

In [ ]:
solutionProcesses = requests.get(base_url + '/solutionProcesses?projectId=' + default_project + '&scopeId=' +  newScopeId + '&$top=10', headers=headers)

print(solutionProcesses.status_code, solutionProcesses.reason)

df = pd.json_normalize(solutionProcesses.json()['value'])

df

* * *
# Set solution processes in scope

Expected response "200 OK"

In [ ]:
def getScopeAssignment(record):
    return {
        'scopeId': newScopeId,
        'solutionScenarioVersionId': record['solutionScenarioVersionId'],
        'solutionProcessVersionId': record['solutionProcessVersionId'],
        'isScoped': True
    }
scopeAssignmentData = { 'value': list(map(getScopeAssignment , solutionProcesses.json()['value'])) } 

updatedScopeAssignments = requests.patch(base_url + '/solutionProcesses/scopeAssignments', headers=patchUpdateHeader, json=scopeAssignmentData)

print(updatedScopeAssignments.status_code, updatedScopeAssignments.reason)
print(updatedScopeAssignments.headers.get('x-correlationid'))

df = pd.json_normalize(updatedScopeAssignments.json()['value'])

df

* * *
# Delete a Scope

Expected response "204 No Content"

In [ ]:
deleteScopeResponse = requests.delete(base_url + '/scopes/' + newScopeId, headers=headers)

print(deleteScopeResponse.status_code, deleteScopeResponse.reason)